In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models import word2vec, Word2Vec
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np
from numpy import argmax
%matplotlib inline

from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


## 입력데이터

In [2]:
df = pd.read_csv('8529_data.csv', sep='\t', encoding='UTF-8')

In [3]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인'],
      dtype='object')

In [4]:
df.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN  
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN  
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN  
3              [C] 현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치     X  NaN  NaN  NaN  
4                     [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고     X  NaN  NaN  NaN

In [5]:
df.shape

(8529, 16)

## 분류현황

In [6]:
category_count = df['구분'].value_counts()
category_count

기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [7]:
category_count = df['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [8]:
category_count = df['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

## 텍스트 전처리

In [9]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
# df['result_pred'] = df['검토결과'].astype(str).copy()
df['target_pred'] = df['대상 선정'].astype(str).copy()
# df['charge_pred'] = df['책임구'].astype(str).copy()
# df['division_pred'] = df['구분'].astype(str).copy()

### (1) 오타, 동의어 처리(replace)

In [10]:
replaceword = pd.read_csv('replaceword_final.csv', sep='\t', encoding='UTF-8')

In [11]:
replaceword.head(10)

Replaceword Missword
0         가니쉬      가니시
1          가스       개스
2          간격       간극
3          간격        갭
4       간격 단차      간단차
5       간격 단차      갭단차
6       간격 단차      단간차
7         간헐적      간혈적
8       감지 센서     감지센서
9       감지 센서    울트라센서

In [12]:
replaceword.shape

(474, 2)

In [13]:
df['clm_prtl_sbc'][488]

'[C] 1.프론트 도어 양쪽 픽싱 클립 장착 부위 좌 우 찢어져 있다고 입고 교환 요청2.육안 점검 및 외부 사고 흔적 없음3.픽싱부위 장착부위 간극 과대4.프론트 도어 사이드 웨더 스트립 어셈블리 좌 우 교환'

In [14]:
Replaceword_list =  replaceword['Replaceword'].tolist()

In [15]:
Missword_list = replaceword['Missword'].tolist()

In [16]:
print(len(Replaceword_list))
print(len(Missword_list))

474
474


In [17]:
for i in range(len(Replaceword_list)):
    df['clm_prtl_sbc'] = df['clm_prtl_sbc'].replace(Missword_list[i], Replaceword_list[i], regex=True)
#regex=True 설정을 주면 조사가 붙어있던 말던 단어를 다 바꿔준다.

In [18]:
df['clm_prtl_sbc'].head()

0    [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...
1                         [C] 1.현상: 리어 도어 안티 치핑 필름 미장착
2    [C] 1.현상:Rr RH 커튼 홀더  미장착 출고됨.2.조치내용:Rr RH 커튼 ...
3             [C] 현상및조치   차량출고시 웨더스트림스트림고무 주름발생 들뜸교환조치
4                       [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고
Name: clm_prtl_sbc, dtype: object

In [19]:
df['clm_prtl_sbc'][488] # '간극'이 '간격'으로 바뀌었는지 확인.(8529 데이터 기준)

'[C] 1.프런트 도어 양쪽 픽싱 클립 장착 부위 좌 우 찢어져 있다고 입고 교환 요청2.육안 점검 및 외부 사고 흔적 없음3.픽싱부위 장착부위 간격 과대4.프런트 도어 사이드 웨더스트림 스트림 어셈블리리 좌 우 교환'

In [20]:
df['clm_prtl_sbc'][670]  # '가니시'가 '가니쉬'로 바뀌었는지 확인.(8529 데이터 기준)

'[C] 1.프런트 운전석 도어 프레임몰딩 단차 발생함2.운전석 도어 프레임 몰딩이 1.5미리 정도 돌출됨3.운전석 도어 섀시 위치 조정 했으나 약간 돌출되어 있음4.가니쉬 내부 양면 테이핑 떨어짐  교환 조치함'

### (2) 사용자 단어 정의(User dictionaray)

In [21]:
udw = pd.read_csv('Userdict_final.csv', sep='\t', encoding='UTF-8') #Userdictionary에 등록할 Word 불러오기.

In [22]:
udw.head()

Userdict
0      가니쉬
1      가벼움
2       가스
3       가죽
4       간격

In [23]:
udw_list =  udw['Userdict'].tolist() # 단어들만 list로 추출.

In [24]:
for word in udw_list: 
    twitter.add_dictionary(word, 'Noun') # for문을 통해 dictionary에 단어들을 Noun으로 추가.

In [25]:
twitter.pos(df['clm_prtl_sbc'][1]) # '리어', '미장착'이 Noun으로 잘 정의되었는지 확인.

[('[', 'Punctuation'),
 ('C', 'Alpha'),
 (']', 'Punctuation'),
 ('1', 'Number'),
 ('.', 'Punctuation'),
 ('현상', 'Noun'),
 (':', 'Punctuation'),
 ('리어', 'Noun'),
 ('도어', 'Noun'),
 ('안티', 'Noun'),
 ('치핑', 'Noun'),
 ('필름', 'Noun'),
 ('미장착', 'Noun')]

### (3) 명사 추출(Twitter)

In [26]:
def nouns(text):
    text = str(text)
    text = twitter.nouns(text)
    return text

In [27]:
df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(nouns)

In [28]:
df.loc[1, 'clm_prtl_sbc']  #'리어', '미장착' 붙어서 출력되었는지 확인.(8529 데이터 기준)

['현상', '리어', '도어', '안티', '치핑', '필름', '미장착']

### (4) 특수문자, 띄어쓰기 전처리

In [29]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    # text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ]', ' ', text) 
    text = re.sub(' +', ' ', text)
    return text

In [30]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['(LH)','(RH)','(사진첨부)','(우)','(좌)','[C]','A/S','C','C-10000','C-10000','F/LH','F/RH','FR','FRT',
             'Frt','frt','L/H','L/H리어','LH','LH/RH','LH쪽','R/H','R/H리어','R/LH','R/RH','RH','RH쪽','Rr','RR','rr',
             '결과','고객','고객에게','공임','광주','그랜저','그랜져','낫','낮','넥쏘','니로','당','당차량','당차량은','대리점',
             '동영상참조','레이','렌트','리무진','리어LH','리어RH','모닝','모하비','밤','벨로스터','사진','사진참조','사진첨부',
             '상기','상기차량은','새벽','새차','서비스','소하','스타렉스','스팅어','스포티지','신차','신차','심야','싼타페',
             '써비스','쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','아침','야간','언덕','엑센트','열','울산','원인',
             '임시 넘버','임시 번호','입고','자동차','저머검내용','점검/원인','점검결과','점검내용','점검및원인','점검및조치내용',
             '점검점검','점검점검내용','조치결과','조치내용','주간','주재원','주제원','주차장','증상','지하','진입','차량',
             '차량은','철도','첨부','첨부파일','청구','출고','카 마스터','카니발','카렌스','코나','클레임','투싼',
             '팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상','현 상','현산','현상','현상및점검',
             '현상및조치','화성'
]
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [31]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

Wall time: 422 ms


In [32]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 537 ms


In [33]:
df['clm_prtl_sbc'].head()

0     엔진룸 다량 연료 누유 발생 점검 엔진룸 다량 연료 누유 점검 점검 내용 엔진룸 ...
1                                  리어 도어 안티 치핑 필름 미장착 
2                        커튼 홀더 미장착 조치 내용 커튼 홀더 장착 후 함 
3                  및 조치 시 웨더 스트림 스트림 무 주름 발생 들뜸 교환 조치 
4                               몰딩 내부 고무 미조립 상태 몰딩 조립 
Name: clm_prtl_sbc, dtype: object

## X, y 정의

In [34]:
from sklearn.preprocessing import LabelEncoder
y_encod = df['대상 선정']
classle = LabelEncoder()
y_encod = classle.fit_transform(df['대상 선정'].values) # y값 인코딩.
print(np.unique(y_encod))

[0 1]


In [35]:
# ro column을 list형태로 ro_text 변수에 할당.
ro_text = list(df['clm_prtl_sbc'])
sentiments = list(y_encod) # y값.

In [36]:
ro_text[0]

' 엔진룸 다량 연료 누유 발생 점검 엔진룸 다량 연료 누유 점검 점검 내용 엔진룸 프런트 저압 라인 연료 공급 파이프 중 고압 펌프 연결 저압 라인 퀵 커넥터 터 오체결 랭킹 중 다량 료 엔진룸 내부 고압 연료 펌프 하단 바닥 누유 발생 함 고압 펌프 저압 라인 퀵 커넥터 터 오체결 연료 펌프 구동 중 이탈 다량 연료 엔진룸 및 다닥 누유 사항 임 조치 내용 고압 펌측 퀵 턴텍터 체결 및 엔진룸 가솔린 오염 부위 세척 시동 후 누유 확인 테스트 요망 사항 해당 저압 파이프 이탈 시 파이프 하단 부위 익 죠스 트 매니폴드 이 어서 가솔린 연료 발화 인하 회재 발생 수 있음 중요 부위 조립 철저 관리 필요 사항 임 '

In [37]:
sentiments[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [38]:
sentences = []
for text in ro_text:
    sentences.append(text.split())

In [39]:
sentences[:3]

[['엔진룸',
  '다량',
  '연료',
  '누유',
  '발생',
  '점검',
  '엔진룸',
  '다량',
  '연료',
  '누유',
  '점검',
  '점검',
  '내용',
  '엔진룸',
  '프런트',
  '저압',
  '라인',
  '연료',
  '공급',
  '파이프',
  '중',
  '고압',
  '펌프',
  '연결',
  '저압',
  '라인',
  '퀵',
  '커넥터',
  '터',
  '오체결',
  '랭킹',
  '중',
  '다량',
  '료',
  '엔진룸',
  '내부',
  '고압',
  '연료',
  '펌프',
  '하단',
  '바닥',
  '누유',
  '발생',
  '함',
  '고압',
  '펌프',
  '저압',
  '라인',
  '퀵',
  '커넥터',
  '터',
  '오체결',
  '연료',
  '펌프',
  '구동',
  '중',
  '이탈',
  '다량',
  '연료',
  '엔진룸',
  '및',
  '다닥',
  '누유',
  '사항',
  '임',
  '조치',
  '내용',
  '고압',
  '펌측',
  '퀵',
  '턴텍터',
  '체결',
  '및',
  '엔진룸',
  '가솔린',
  '오염',
  '부위',
  '세척',
  '시동',
  '후',
  '누유',
  '확인',
  '테스트',
  '요망',
  '사항',
  '해당',
  '저압',
  '파이프',
  '이탈',
  '시',
  '파이프',
  '하단',
  '부위',
  '익',
  '죠스',
  '트',
  '매니폴드',
  '이',
  '어서',
  '가솔린',
  '연료',
  '발화',
  '인하',
  '회재',
  '발생',
  '수',
  '있음',
  '중요',
  '부위',
  '조립',
  '철저',
  '관리',
  '필요',
  '사항',
  '임'],
 ['리어', '도어', '안티', '치핑', '필름', '미장착'],
 ['커튼', '홀더', '미장착', '조치', '내용', '커튼', 

## Word2Vec

In [40]:
# Word2Vec의 파라미터 값들 정의.
num_features = 300    
min_word_count = 40   
num_workers = 4       
context = 10          
downsampling = 1e-3 

In [41]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
   level=logging.INFO)

In [42]:
# word2vec으로 단어 학습.
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers, \
           size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2019-06-03 15:11:41,928 : INFO : collecting all words and their counts
2019-06-03 15:11:41,928 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-06-03 15:11:41,975 : INFO : collected 6302 word types from a corpus of 229749 raw words and 8529 sentences
2019-06-03 15:11:41,976 : INFO : Loading a fresh vocabulary
2019-06-03 15:11:41,981 : INFO : effective_min_count=40 retains 608 unique words (9% of original 6302, drops 5694)
2019-06-03 15:11:41,982 : INFO : effective_min_count=40 leaves 202100 word corpus (87% of original 229749, drops 27649)
2019-06-03 15:11:41,984 : INFO : deleting the raw counts dictionary of 6302 items
2019-06-03 15:11:41,985 : INFO : sample=0.001 downsamples 70 most-common words
2019-06-03 15:11:41,985 : INFO : downsampling leaves estimated 126000 word corpus (62.3% of prior 202100)
2019-06-03 15:11:41,987 : INFO : estimated required memory for 608 words and 300 dimensions: 1763200 bytes
2019-06-03 15:11:41,987 : INFO : resetting layer 

In [43]:
# Word2Vec의 feature vector를 얻는 함수.
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [44]:
def get_dataset(ro_text, model, num_features):
    dataset = list()

    for s in ro_text:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [45]:
test_data_vecs = get_dataset(sentences, model, num_features)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: invalid value encountered in true_divide
  del sys.path[0]


In [46]:
test_data_vecs

array([[-0.0670774 , -0.04729139,  0.2485065 , ...,  0.16027486,
        -0.09168445, -0.06247993],
       [-0.04631535,  0.04234431,  0.06145272, ..., -0.24849665,
         0.37317786,  0.23887159],
       [-0.17617406, -0.09830552,  0.15528554, ..., -0.00614598,
         0.10057976,  0.03864539],
       ...,
       [-0.09926555, -0.17096522,  0.11474898, ...,  0.14744836,
         0.02083559, -0.09822653],
       [ 0.14025025, -0.03069651,  0.1177343 , ...,  0.15146366,
         0.04116388, -0.01456596],
       [ 0.04136943, -0.19660191,  0.14953879, ...,  0.29104334,
        -0.05126983, -0.15363722]], dtype=float32)

In [47]:
test_data_vecs[2]

array([-0.17617406, -0.09830552,  0.15528554,  0.02736155,  0.2884354 ,
        0.0283219 ,  0.15840684, -0.08759727, -0.23840901, -0.20519817,
       -0.24019039, -0.10480763, -0.02308775,  0.01501108,  0.15072674,
       -0.05343766, -0.26366597,  0.11695822, -0.02807054, -0.11512253,
        0.11543293,  0.02387893,  0.08222236, -0.00560115, -0.07523459,
        0.21581432,  0.06761812,  0.02871771, -0.10625094, -0.03123594,
       -0.31002668,  0.00763384, -0.12257389,  0.07130183,  0.02787882,
       -0.10805936, -0.06886143, -0.01986719, -0.03435279, -0.01821187,
        0.06049429,  0.18805404, -0.13068876,  0.1279936 ,  0.01678888,
       -0.12240168,  0.02387247, -0.00743456, -0.0095228 , -0.31511647,
        0.05068328, -0.07952566, -0.20551379,  0.14664178,  0.0345571 ,
       -0.11518393,  0.09387834, -0.04626315,  0.07991905, -0.34926537,
       -0.10898107,  0.01348209, -0.13516566, -0.04639009,  0.09680903,
       -0.2277708 ,  0.05172287, -0.09405197, -0.03859969,  0.05

## Train, Test data Split

In [48]:
from sklearn.model_selection import train_test_split
import numpy as np

X = test_data_vecs
y = np.array(sentiments)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6823, 300)
(1706, 300)
(6823,)
(1706,)


In [50]:
np.where(np.isnan(X_train))

(array([2305, 2305, 2305, ..., 6357, 6357, 6357], dtype=int64),
 array([  0,   1,   2, ..., 297, 298, 299], dtype=int64))

In [51]:
np.where(np.isnan(X_test))

(array([], dtype=int64), array([], dtype=int64))

In [52]:
np.where(np.isnan(y_train))

(array([], dtype=int64),)

In [53]:
np.where(np.isnan(y_test))

(array([], dtype=int64),)

In [54]:
np.where(np.isnan(X_train))

(array([2305, 2305, 2305, ..., 6357, 6357, 6357], dtype=int64),
 array([  0,   1,   2, ..., 297, 298, 299], dtype=int64))

In [55]:
X_train[2305]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, na

In [56]:
# 나중에 모델에 넣을 때 에러가 나기때문에 nan인 값들 처리 필요.
X_nan = np.where(np.isnan(X_train))
X_nan

(array([2305, 2305, 2305, ..., 6357, 6357, 6357], dtype=int64),
 array([  0,   1,   2, ..., 297, 298, 299], dtype=int64))

In [57]:
type(X_nan)

tuple

In [58]:
X_nan_list = list(X_nan)

In [59]:
X_set =set(X_nan_list[0])
X_set

{2305, 2651, 3253, 3720, 6357}

In [60]:
X_set_list = list(X_set)
X_set_list

[2305, 3720, 3253, 6357, 2651]

In [61]:
for a in X_set_list:
    X_train[a] = np.nan_to_num(X_train[a])  

In [62]:
np.where(np.isnan(X_train))

(array([], dtype=int64), array([], dtype=int64))

In [63]:
# 잘 변경되었는지 확인.
X_train[2305]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

## RandomForestClassifier로 학습

In [64]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=42)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [65]:
%time forest = forest.fit(X_train, y_train)

Wall time: 1.77 s


## 모델 평가

In [66]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, X_train, y_train, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 9.37 s


array([0.81758242, 0.81098901, 0.82124542, 0.80571848, 0.79618768])

In [67]:
round(np.mean(score)*100,2)

81.03

## 예측 결과

In [68]:
from sklearn.metrics import accuracy_score
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(X_test)
y_pred[:3]

array([1, 1, 1])

In [69]:
print(accuracy_score(y_test, y_pred))

0.805978898007034
